In [ ]:
import numpy as np
import xarray as xr
import dask
import xgcm
import postmit as pm
import seawater as sw

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

### Heat transport on isopycnals

The meridional HT in isopycnal coordinates is computed following Lee et al., 2007. The velocity $v$ and temperature $T$ are mapped to isopycnal layers with thickness $h$, resulting in $v_{\sigma}$ and $T_{\sigma}$, and the total, time-averaged temperature transport can be decomposed such that

$\overline{v_{\sigma}T_{\sigma}h} = \overline{v}_{eu}\,\hat{T_{\sigma}}\,\overline{h} + \overline{v}_{eddy}\,\hat{T_{\sigma}}\,\overline{h} + \widehat{v_{\sigma}''\,T_{\sigma}''}\,\overline{h}$


where $v_{eu}$ is the time-averaged meridional velocity in z-coordinates  mapped to mean isopycnal layers, the $\hat{\cdot}=\frac{\overline{(\cdot\,h)}}{\overline{h}}$ is the layer thickness-weighted time average, the double-prime denotes a deviation thereof, and $v_{eddy} = \hat{v}_{\sigma} - \overline{v}_{eu}$. The first term on the rhs is the mean temperature advection, the second term is the advection of temperature due to transient processes and the third term represents the diffusion by the transient processes. The advective component of the transient HT on isopycnals is thus

$THT_{iso}^{adv}(y, r) = \rho C_{p} [\overline{v}_{eddy}\,\hat{T}_{\sigma}\,\overline{h}]_{\sigma} = \rho C_{p} [\frac{\overline{v_{\sigma}h}}{\overline{h}}\,\overline{T_{\sigma} h} - \overline{v_{eu}}\,\overline{T_{\sigma} h}]_{\sigma}$

where $r$ is the density-coordinate and $[\cdot]_{\sigma}$ indicates a zonal average along isopycnals. The diffusive part is represented by

$THT_{iso}^{diff}(y, r) = \rho C_{p} [\widehat{v_{\sigma}''\,T_{\sigma}''}\,\overline{h}]_{\sigma} = \rho C_{p} [\overline{(v_{\sigma} - \frac{\overline{v_{\sigma} h}}{\overline{h}}) (T_{\sigma} - \frac{\overline{T_{\sigma} h}}{\overline{h}})\,h}]_{\sigma}$

and the total transient, along-isopycnal HT is computed as

$THT_{iso}(y, r) = THT_{iso}^{adv}(y, r) + THT_{iso}^{diff}(y, r)$


To compute the contribution by coherent mesoscale eddies to the along-isopycnal heat transport, we again use the eddymasks:

$THT_{iso}^{CME,adv}(y, r) = \rho C_{p} [\frac{\overline{v_{\sigma} M^{CME} h}}{\overline{h}}\,\overline{T_{\sigma} h} - \overline{v_{eu}}\,\overline{T_{\sigma} h}]_{\sigma}$
 
and

$THT_{iso}^{CME,diff}(y, r) = \rho C_{p} [\overline{(v_{\sigma} M^{CME} - \frac{\overline{v_{\sigma} M^{CME} h}}{\overline{h}}) (T_{\sigma} - \frac{\overline{T_{\sigma} h}}{\overline{h}})\,h}]_{\sigma}$

The total $THT_{iso}^{CME}$ is computed equivalently to before as

$THT_{iso}^{CME}(y, r) = THT_{iso}^{CME,adv}(y, r) + THT_{iso}^{CME,diff}(y, r)$


### Helper functions

`convert2Zp1` interpolates a variable to the grid cell's faces. I.e. in MITgcm terms, from `Z` to `Zp1`. This is needed for the remapping to sigma-coordinates.

In [ ]:
def convert2Zp1(var, grid):
    part1 = grid.interp(var, "Z", to="left", boundary="extend").rename({"Zl": "Zp1"})
    part2 = grid.interp(var, "Z", to="right", boundary="extend").isel(Zu=slice(-1, None)).rename({"Zu": "Zp1"})
    return xr.concat((part1, part2), dim="Zp1")

`param_and_prepare` sets parameters needed for the calculations and does som pre-processing.

In [ ]:
def param_and_prepare(data):
    # set constants
    rho = 1035.
    Cp = 3994.
    # sigma-levels to map to later
    target_rho_levelsC = np.hstack(([5., 15., 22.5, 27.5, 30.5, 31.5], 
                                    np.arange(32.05, 35., 0.1),
                                    np.arange(35.025, 36., 0.05),
                                    np.arange(36.01, 36.9, 0.02),
                                    np.arange(36.905, 37.2, 0.01),
                                    np.arange(37.21, 37.6, 0.02),
                                    np.arange(37.605, 37.8, 0.01),
                                    np.arange(37.825, 37.9, 0.05),
                                    [37.95, 38.05, 38.2, 38.4, 38.75, 39.5, 42.5, 47.5, 75.]))
    target_rho_levelsB = np.hstack(([0., 10., 20., 25., 30., 31.], 
                                    np.arange(32., 35., 0.1),
                                    np.arange(35., 36., 0.05),
                                    np.arange(36., 36.9, 0.02),
                                    np.arange(36.9, 37.199, 0.01),
                                    np.arange(37.2, 37.6, 0.02),
                                    np.arange(37.6, 37.8, 0.01),
                                    np.arange(37.8, 37.899, 0.05),
                                    [37.9, 38., 38.1, 38.3, 38.5, 39., 40., 45., 50., 100.]))
    # timestamp for the files to write to disk
    savetime = data.time.isel(time=slice(int(len(data.time) / 2), int(len(data.time) / 2) + 1))
    # compute the reference temperature (freezing point)
    tref = sw.eos80.fp(data.SALT, 1.065)
    # metrics of the Z-axis for the xgcm grid instance
    metrics = {
        ('Z'): ['drC', 'drF', 'drS', 'drW'], # Z distances
    }
    grid = xgcm.Grid(data, periodic=["X"], metrics=metrics)
    # calculate sigma_2
    data = pm.calcs.sigi(data, 2)  
    sigName = "SIG2"
    # define dummy variables with sizes as THETA, VVEL etc. filled with ones
    ones_gridT = (data.THETA / data.THETA)
    ones_gridV = (data.VVEL / data.VVEL)
    ones_gridZ = (grid.interp(data.VVEL, "X", boundary="extend") / grid.interp(data.VVEL, "X", boundary="extend"))
    # define each levels (in depth-space) thickness
    data["levThick_all"] = (data.drF * data.hFacC * data.maskC).expand_dims(dim={"time": data.time})
    # convert sigma_2 to cell faces (on Z-axis)
    data["SIG2"] = (data.SIG2 - 1000.).where(data.maskC == 1)
    data["SIG2B"] = convert2Zp1(data.SIG2, grid).chunk({"time": 72, "Zp1": 56, "YC": 320, "XC": 240})
    # do the same for sigma_2 interpolated to V-points
    data["SIG2v"] = grid.interp(data.SIG2, "Y", boundary="extend").rename("SIG2")
    data["SIG2vB"] = convert2Zp1(data.SIG2v, grid).chunk({"time": 72, "Zp1": 56, "YG": 320, "XC": 240})
    # and again for sigma_2 interpolate to F-points
    data["SIG2z"] = grid.interp(data.SIG2v, "X", boundary="extend").rename("SIG2")
    data["SIG2zB"] = convert2Zp1(data.SIG2z, grid).chunk({"time": 72, "Zp1": 56, "YG": 320, "XG": 240})
    # define a mask that is "1" within the mean winter mixed layer and "0" elsewhere
    data["xMLD"] = ones_gridT.where(
        -data.Z > data.MXLDEPTH.groupby("time.season").mean("time").mean("XC").sel(season="JJA")
        ) 
    # get this mask also on the cell faces, V-points and F-points
    data["xMLDB"] = convert2Zp1(data.xMLD, grid).chunk({"time": 72, "Zp1": 56, "YC": 320, "XC": 240})
    data["xMLDv"] = ones_gridV.where(
        -data.Z > grid.interp(data.MXLDEPTH.groupby("time.season").mean("time").mean("XC").sel(season="JJA"), 
                              "Y", boundary="extend")
        )
    data["xMLDz"] = ones_gridZ.where(
        -data.Z > grid.interp(data.MXLDEPTH.groupby("time.season").mean("time").mean("XC").sel(season="JJA"), 
                              "Y", boundary="extend")
        )
    return rho, Cp, target_rho_levelsC, target_rho_levelsB, savetime, tref, sigName, data, grid

`z2rho` maps a variable defined on z-levels to the given isopycnal levels.

In [ ]:
def z2rho(grid, var, target_levels, target_data, target_name, method):
    var_rho = grid.transform(var, "Z", target_levels, target_data=target_data, method=method)
    var_rho = var_rho.rename({target_name: "layer_center"})
    return var_rho

`calc_adviso` and `calc_adviso_eddy` are the functions calculating 

$THT_{iso}^{adv}(y, r) = \rho C_{p} [\overline{v}_{eddy}\,\hat{T}_{\sigma}\,\overline{h}]_{\sigma} = \rho C_{p} [\frac{\overline{v_{\sigma}h}}{\overline{h}}\,\overline{T_{\sigma} h} - \overline{v_{eu}}\,\overline{T_{\sigma} h}]_{\sigma}$

and 

$THT_{iso}^{CME,adv}(y, r) = \rho C_{p} [\frac{\overline{v_{\sigma} M^{CME} h}}{\overline{h}}\,\overline{T_{\sigma} h} - \overline{v_{eu}}\,\overline{T_{\sigma} h}]_{\sigma}$

In [ ]:
def calc_adviso(t, v, v_eu, h, h_bar, dx, grid, rho, Cp, season):
    # calculate bar(T h) on F-points
    thZ = grid.interp((t * h.data).groupby('time.month').mean("time"), ("X", "Y"), boundary="extend")
    # get h and bar(h) on V-points
    hV = grid.interp(h, "Y", boundary="extend")
    h_barV = grid.interp(h_bar, "Y", boundary="extend")
    # compute bar(v h) / bar(h) on F-points
    vh_hZ = grid.interp((v * hV.data).groupby('time.month').mean("time") / h_barV.data, ("X"), boundary="extend")
    vh_hZ = vh_hZ.where(~np.isnan(vh_hZ), other=0)
    # get v_eu on F-points
    v_euZ = grid.interp(v_eu, ("X"), boundary="extend")
    # compute THT^{adv}_{iso}
    tht = (rho * Cp * ((thZ * vh_hZ) - (thZ * v_euZ)) * dx).sum("XG")
    # averaging over the whole seasonal cycle or just one season
    if season==0:
        out = tht.mean("month")
    elif season=="DJF":
        out = tht.sel(month=[1, 2, 12]).mean("month")
    elif season=="MAM":
        out = tht.sel(month=[3, 4, 5]).mean("month")
    elif season=="JJA":
        out = tht.sel(month=[6, 7, 8]).mean("month")
    elif season=="SON":
        out = tht.sel(month=[9, 10, 11]).mean("month")
    return out

In [ ]:
def calc_adviso_eddy(t, v, v_eu, h, h_bar, dx, season):
    # get h on F-points
    hZ = grid.interp(h, ("X", "Y"), boundary="extend").data
    # calculate bar(T h) on F-points
    thZ = (t * hZ).groupby('time.month').mean("time")
    # compute bar(v h) / bar(h) on F-points
    vh_hZ = (v * hZ).groupby('time.month').mean("time") / grid.interp(h_bar, ("X", "Y"), boundary="extend").data
    vh_hZ = vh_hZ.where(~np.isnan(vh_hZ), other=0)
    # get v_eu on F-points
    v_euZ = grid.interp(v_eu, ("X"), boundary="extend")
    # compute THT^{CME,adv}_{iso}
    tht = (rho * Cp * ((thZ * vh_hZ) - (thZ * v_euZ)) * dx).sum("XG")
    # averaging over the whole seasonal cycle or just one season
    if season==0:
        out = tht.mean("month")
    elif season=="DJF":
        out = tht.sel(month=[1, 2, 12]).mean("month")
    elif season=="MAM":
        out = tht.sel(month=[3, 4, 5]).mean("month")
    elif season=="JJA":
        out = tht.sel(month=[6, 7, 8]).mean("month")
    elif season=="SON":
        out = tht.sel(month=[9, 10, 11]).mean("month")
    return out

`calc_diffiso` and `calc_diffiso_eddy` compute  

$THT_{iso}^{diff}(y, r) = \rho C_{p} [\widehat{v_{\sigma}''\,T_{\sigma}''}\,\overline{h}]_{\sigma} = \rho C_{p} [\overline{(v_{\sigma} - \frac{\overline{v_{\sigma} h}}{\overline{h}}) (T_{\sigma} - \frac{\overline{T_{\sigma} h}}{\overline{h}})\,h}]_{\sigma}$

and 

$THT_{iso}^{CME,diff}(y, r) = \rho C_{p} [\overline{(v_{\sigma} M^{CME} - \frac{\overline{v_{\sigma} M^{CME} h}}{\overline{h}}) (T_{\sigma} - \frac{\overline{T_{\sigma} h}}{\overline{h}})\,h}]_{\sigma}$

In [ ]:
def calc_diffiso(t, v, h, h_bar, dx, grid, rho, Cp, season):
    # get h on V-points
    hV = grid.interp(h, "Y", boundary="extend")
    # get h on F-points
    hZ = grid.interp(h, ("X", "Y"), boundary="extend")
    # get bar(h) on V-points
    h_barV = grid.interp(h_bar, "Y", boundary="extend")
    # get v on F-points
    vZ = grid.interp(v, "X", boundary="extend").groupby("time.month")
    # compute bar(v h) / bar(h) on F-points
    vh_hZ = grid.interp((v * hV.data).groupby('time.month').mean("time") / h_barV.data, ("X"), boundary="extend")
    vh_hZ = vh_hZ.where(~np.isnan(vh_hZ), other=0)
    # get T on F-points
    tZ = grid.interp(t, ("X", "Y"), boundary="extend").groupby("time.month")
    # compute bar(T h) / bar(T) on F-points
    th_hZ = grid.interp((t * h.data).groupby('time.month').mean("time") / h_bar.data, ("X", "Y"), boundary="extend")
    th_hZ = th_hZ.where(~np.isnan(th_hZ), other=0)
    # compute THT^{diff}_{iso}
    tht = (rho * Cp * ((vZ - vh_hZ) * (tZ - th_hZ) * hZ.data) * dx).sum("XG")
    # averaging over the whole seasonal cycle or just one season
    if season==0:
        out = tht.mean("time")
    else:
        out = tht.groupby("time.season").mean("time").sel(season=season)
    return out

In [ ]:
def calc_diffiso_eddy(t, v, v_eddy, h, h_bar, dx, grid, rho, Cp, season):
    # get h on V-points
    hV = grid.interp(h, "Y", boundary="extend")
    # get h on F-points
    hZ = grid.interp(h, ("X", "Y"), boundary="extend")
    # get bar(h) on F-points
    h_barZ = grid.interp(h_bar, ("X", "Y"), boundary="extend")
    # get v M^CME
    vZ_eddy = v_eddy.groupby("time.month")
    # compute bar(v M^CME h) / bar(h) on F-points
    vh_hZ_eddy = (v_eddy * hZ.data).groupby('time.month').mean("time") / h_barZ.data
    vh_hZ_eddy = vh_hZ_eddy.where(~np.isnan(vh_hZ_eddy), other=0)
    # get T on F-points
    tZ = grid.interp(t, ("X", "Y"), boundary="extend").groupby("time.month")
    # compute bar(T h) / bar(T) on F-points
    th_hZ = grid.interp((t * h.data).groupby('time.month').mean("time") / h_bar.data, ("X", "Y"), boundary="extend")
    th_hZ = th_hZ.where(~np.isnan(th_hZ), other=0)
    # compute THT^{CME,diff}_{iso}
    tht = (rho * Cp * ((vZ_eddy - vh_hZ_eddy) * (tZ - th_hZ) * hZ.data) * dx).sum("XG")
    # averaging over the whole seasonal cycle or just one season
    if season==0:
        out = tht.mean("time")
    else:
        out = tht.groupby("time.season").mean("time").sel(season=season)
    return out

## Main function
This function takes care of applying the previously defined ones to compute the different components of isopycnal heat transport following Leet et al., 2007.

In [ ]:
def lee_eddy(data, em, em_anti, em_cyc, grid, sigma_name, rho, Cp, tref, target_rho_levelsC, target_rho_levelsB, excludeMLD=0, season=0):
    # depending on whether we want to exclude the mixed layer or include only the mixed layer, we multiply the variables with the
    # previously defined masks
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        if excludeMLD == 1:
            xMLD = data["xMLD"]
            xMLDv = data["xMLDv"]
            xMLDb = data["xMLDB"]
            T = (data.THETA - tref) * xMLD
            v = data.VVEL * xMLDv
            sigma = (data.SIG2 * xMLD).rename(sigma_name)
            sigmaV = (data.SIG2v * xMLDv).rename(sigma_name)
            sigmaB = (data.SIG2B * xMLDb).rename(sigma_name)
            monthly_sigmaV = (data.SIG2v * xMLDv).groupby('time.month').mean("time").rename(sigma_name)
            levThick = data.levThick_all * xMLD
        elif excludeMLD == -1:
            MLD = data["xMLD"].fillna(1).where(((np.isnan(data["xMLD"])) & (data.THETA != 0)))
            MLDv = data["xMLDv"].fillna(1).where(((np.isnan(data["xMLDv"])) & (data.VVEL != 0)))
            MLDb = data["xMLDB"].fillna(1).where(((np.isnan(data["xMLDB"])) & (data.SIG2B != 0)))
            T = (data.THETA - tref) * MLD
            v = data.VVEL * MLDv
            sigma = (data.SIG2 * MLD).rename(sigma_name)
            sigmaV = (data.SIG2v * MLDv).rename(sigma_name)
            sigmaB = (data.SIG2B * MLDb).rename(sigma_name)
            monthly_sigmaV = (data.SIG2v * MLDv).groupby('time.month').mean("time").rename(sigma_name)
            levThick = data.levThick_all * MLD
        else:
            T = (data.THETA - tref)
            v = data.VVEL
            sigma = (data.SIG2).rename(sigma_name)
            sigmaV = (data.SIG2v).rename(sigma_name)
            sigmaB = (data.SIG2B).rename(sigma_name)
            monthly_sigmaV = data.SIG2v.groupby('time.month').mean("time").rename(sigma_name)
            levThick = data.levThick_all
        # transform t and v to isopycnals
        t_rho = z2rho(grid, T, target_rho_levelsC, sigma, sigma_name, "linear").chunk({"time": 72, "YC": 320, "XC": 240})
        t_rho = t_rho.where(~np.isnan(t_rho), other=0)
        # need t_rho on F-points as well
        t_rhoZ = grid.interp(t_rho, ("X", "Y"), boundary="extend")
        v_rho = z2rho(grid, v, target_rho_levelsC, sigmaV, sigma_name, "linear").chunk({"time": 72, "YG": 320, "XC": 240})
        v_rho = v_rho.where(~np.isnan(v_rho), other=0)
        # multiply v_rho with the different eddymasks to calculate the CME contribution
        v_rho_eddy = (grid.interp(v_rho, ("X"), boundary="extend") * em).chunk({"time": 72, "YG": 320, "XG": 240})
        v_rho_eddy = v_rho_eddy.where(~np.isnan(v_rho_eddy), other=0)
        v_rho_eddy_cyc = (grid.interp(v_rho, ("X"), boundary="extend") * em_cyc).chunk({"time": 72, "YG": 320, "XG": 240})
        v_rho_eddy_cyc = v_rho_eddy_cyc.where(~np.isnan(v_rho_eddy_cyc), other=0)
        v_rho_eddy_anti = (grid.interp(v_rho, ("X"), boundary="extend") * em_anti).chunk({"time": 72, "YG": 320, "XG": 240})
        v_rho_eddy_anti = v_rho_eddy_anti.where(~np.isnan(v_rho_eddy_anti), other=0)
        # get level thicknesses of isopycnals 
        levThick_rho = z2rho(grid, levThick, target_rho_levelsB, sigmaB, sigma_name, 
                             "conservative").chunk({"time": 72, "YC": 320, "XC": 240})
        levThick_rho = levThick_rho.where(~np.isnan(levThick_rho.data), other=0)
        levThickbar_rho = levThick_rho.groupby('time.month').mean("time")
        levThick_rho_all = z2rho(grid, data.levThick_all, target_rho_levelsB, sigmaB, sigma_name, 
                                 "conservative").chunk({"time": 72, "YC": 320, "XC": 240})
        levThick_rho_all = levThick_rho_all.where(~np.isnan(levThick_rho_all.data), other=0)
        levThickbar_rho_all = levThick_rho_all.groupby('time.month').mean("time")
        # compute v_eu (eulerian mean v transformed to rho-space)
        v_eu_z = v.groupby('time.month').mean("time")
        v_eu = z2rho(grid, v_eu_z, target_rho_levelsC, monthly_sigmaV, sigma_name, "linear")
        v_eu = v_eu.where(~np.isnan(v_eu), other=0) 
        # compute advective part of transient ispycnal heat transport
        adviso = calc_adviso(t_rho, v_rho, v_eu, levThick_rho, levThickbar_rho, data.dxV, grid, rho, Cp, season)
        adviso_eddy = calc_adviso_eddy(t_rhoZ, v_rho_eddy, v_eu, levThick_rho, levThickbar_rho, data.dxV, season)
        adviso_eddy_cyc = calc_adviso_eddy(t_rhoZ, v_rho_eddy_cyc, v_eu, levThick_rho, levThickbar_rho, data.dxV, season)
        adviso_eddy_anti = calc_adviso_eddy(t_rhoZ, v_rho_eddy_anti, v_eu, levThick_rho, levThickbar_rho, data.dxV, season)
        # compute diffusive part of transient isopycnal heat transport
        diffiso = calc_diffiso(t_rho, v_rho, levThick_rho, 
                               levThickbar_rho, data.dxV, grid, rho, Cp, season)
        diffiso_eddy = calc_diffiso_eddy(t_rho, v_rho, v_rho_eddy,
                                         levThick_rho, levThickbar_rho, data.dxV, grid, rho, Cp, season)
        diffiso_eddy_cyc = calc_diffiso_eddy(t_rho, v_rho, v_rho_eddy_cyc,
                                             levThick_rho, levThickbar_rho, data.dxV, grid, rho, Cp, season)
        diffiso_eddy_anti = calc_diffiso_eddy(t_rho, v_rho, v_rho_eddy_anti, 
                                              levThick_rho, levThickbar_rho, data.dxV, grid, rho, Cp, season)
        # create dataset as output
        THTiso_lee = adviso + diffiso
        HTiso = xr.Dataset({"THTiso_lee": THTiso_lee, 
                            "ADViso": adviso,
                            "ADViso_eddy": adviso_eddy,
                            "ADViso_eddy_anticyclones": adviso_eddy_anti,
                            "ADViso_eddy_cyclones": adviso_eddy_cyc,
                            "DIFFiso": diffiso,
                            "DIFFiso_eddy": diffiso_eddy,
                            "DIFFiso_eddy_cyclones": diffiso_eddy_cyc,
                            "DIFFiso_eddy_anticyclones": diffiso_eddy_anti,
                            "levThick": (("YG", "layer_center"), 
                                          grid.interp(levThickbar_rho_all.mean(("XC", "month")), 
                                                      "Y", boundary="extend").data[:, :])})
    return HTiso

## Loop over decades

We compute the heat transports for each decade of the 100 years.

In [ ]:
t1 = np.arange(201, 300, 10)
t2 = np.arange(210, 301, 10)
for a, b in zip(t1, t2):
    y1 = "0" + str(a)
    y2 = "0" + str(b)
    print(y1 + " - " + y2)
    # loading the data and the eddymasks
    data = xr.open_zarr(path + "zarr_Diags/output.5d.zarr").sel(time=slice(y1 + "-01-01", y2 + "-12-30"))
    em = xr.open_mfdataset(eddypath + 'eddymask_0201-0300.nc').eddymask.squeeze().rename({"lon": "XG", "lat": "YG"})
    em_cyc = xr.open_mfdataset(eddypath + 'eddymask_cyclones_0201-0300.nc').eddymask_cyclones.squeeze().rename({"lon": "XG", "lat": "YG"})
    em_anti = xr.open_mfdataset(eddypath + 'eddymask_anticyclones_0201-0300.nc').eddymask_anticyclones.squeeze().rename({"lon": "XG", "lat": "YG"})
    # pre-process data and get constants and parameters
    rho, Cp, targetC, targetB, savetime, tref, sigName, data, grid = param_and_prepare(data)
    data = data.drop("season")
    # loop over mean seasonal cycle (season=0) and the four seasons
    seasons = [0, "DJF", "MAM", "JJA", "SON"]
    for season in seasons:
        if season == 0:
            print("annual mean")
            s = "."
        else:
            print(season)
            s = "." + season + "."
        HTiso = lee_eddy(data, em, em_anti, em_cyc, grid, sigName, rho, Cp, tref, targetC, targetB, excludeMLD=0, season=season)
        HTiso = HTiso.expand_dims({"time": savetime}).chunk({"time": 1, "YG": 320, "layer_center": 182})
        HTiso.to_netcdf(path + "post/HTiso_eddy" + s + y1 + "_" + y2 + ".nc")
        ## Exlude the Mixed Layer   
        HTisoNoMLjja = lee_eddy(data, em, em_anti, em_cyc, grid, sigName, rho, Cp, tref, targetC, targetB, excludeMLD=1, season=season)
        HTisoNoMLjja = HTisoNoMLjja.expand_dims({"time": savetime}).chunk({"time": 1, "YG": 320, "layer_center": 182})
        HTisoNoMLjja.to_netcdf(path + "post/HTisoNoMLjja_eddy." + s + y1 + "_" + y2 + ".nc")
        ## Only Mixed Layer
        HTisoMLjja = lee_eddy(data, em, em_anti, em_cyc, grid, sigName, rho, Cp, tref, targetC, targetB, excludeMLD=-1, season=season)
        HTisoMLjja = HTisoMLjja.expand_dims({"time": savetime}).chunk({"time": 1, "YG": 320, "layer_center": 182})
        HTisoMLjja.to_netcdf(path + "post/HTisoMLjja_eddy." + s + y1 + "_" + y2 + ".nc")